本次分享主要内容是大模型最核心的地方：自注意力机制。
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image001.png)

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image003.png)

接下来会分四部分，第一部分是自注意力机制简化版，第二部分是自注意力机制，第三部分是因果注意力(causal attention)，第四部分是多头注意力(multi-head attention)。

#### 1.The problem with modeling long sequences
让咱们看下在没有自注意力机制之前如何解决长序列问题的，这里咱们拿语言翻译举例。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image005.png)

上图是德文翻译成英文的例子，发现单词不能逐词翻译，不然就会乱套，因为每种语言的语法不一样的，这需要模型能够看到上下文，才能做出正确的翻译。</br>

那么自然想到的办法是使用encoder-decoder架构，先使用encoder看到所有输入，理解上下文，然后使用decoder逐词输出。</br>

在transformer架构出来之前，最流行的encoder-decoder架构是RNN，比如LSTM和GRU。


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image007.png)

如上图所示，所有的输入先经过RNN encoder，得到一个隐状态，在encoder里，输出是下一步的输入，所以隐状态是包含了上下文的。然后隐状态作为decoder的输入。

可以简单把hidden state理解为包含了上下文的embedding vector。

RNN的缺点是，decoder无法获取之前的隐状态信息，所以需要把所有的隐状态都保存下来，这样会占用大量内存。同时长距离文本处理起来比较困难。



#### 2.Capturing data dependencies with self-attention

2014年，研究人员提出了一个Bahdanau Attention的机制，通过修改encoder-decoder架构，以便decoder能够有选择的获得输入序列的不同部分。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image009.png)

如上图，在decoder阶段，针对特定输出，可以获得输入序列的不同部分，并通过attention weight来衡量输入序列不同部分对当前输出的重要性。

Bahdanau Attention机制的缺点是，无法并行处理，在encoder阶段，所有的输入必须一个一个处理，在decoder阶段，当前时刻的隐状态依赖于前一个时刻的隐状态，所以也无法并行处理。

三年后2017年，受到了Bahdanau Attention的启发，提出了transformer架构，使用自注意力机制来解决长距离依赖问题。

Self-attention（自注意力）是一种让输入序列中的每个位置都能够在计算序列表示时关注同一序列中所有位置的机制。这种机制使得模型能够捕捉序列内部的长距离依赖关系，并且理解不同位置之间的复杂关系。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image011.png)

#### 3.Attending to different parts of the input with self-attention

本章会介绍自注意力机制内部工作机制，以及学习如何从头实现自注意力机制。自注意力机制作为大模型的基石，非常值得花费时间去理解，一旦掌握了机制背后的原理，对后续的深入学习非常有帮助。

self-attention里的self表示什么意思呢？

“Self”在自注意力中指的是机制能够计算不同位置之间的注意力权重，这些位置都位于单个输入序列内部。这意味着模型在处理序列时，会分析序列内部各部分之间的关系，例如句子中的单词或图像中的像素。

自注意力机制评估并学习输入序列内部各部分之间的关系和依赖性。这种机制使得模型能够理解序列中各个元素之间的相互作用，而不仅仅是序列作为一个整体与其他序列之间的关系。

传统的注意力机制通常关注两个不同序列之间的元素关系，例如在序列到序列（sequence-to-sequence）模型中，注意力可能集中在输入序列和输出序列之间的关系。这种注意力机制在机器翻译等任务中很常见，其中模型需要关注输入序列（如一种语言的文本）与输出序列（如另一种语言的文本）之间的关系，比如上图所示。

自注意力在自然语言处理（NLP）和计算机视觉（CV）等领域有广泛应用。在NLP中，它可以用于理解文本中单词之间的关系；在CV中，它可以用于理解图像中不同像素或区域之间的关系。

自注意力机制比较复杂，为了让大家更方便的理解其原理，我们先实现一个简单的没有可训练权重的自注意力机制，然后逐步增加可训练权重，最后实现一个完整的自注意力机制。


##### 3.1 A simple self-attention mechanism without trainable weights

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image013.png)

自注意力机制的目标是为输入序列中的每个元素计算一个上下文向量(context vector),这个向量结合了来自序列中所有其他元素的信息。假如咱们要对第二个元素${x^{(2)}}$计算上下文向量${z^{(2)}}$,每个输入元素对于计算上下文向量${z_2}$的重要性或者贡献度由注意力权重${a_{21}}$到${a_{2T}}$决定。这里的T表示输入序列的长度，而${a_{2i}}$表示第i个元素对第2个元素的注意力权重。上下文向量${z^{(2)}}$的计算公式如下：

$$z^{(2)} = \sum_{i=1}^{T} a_{2i} x^{(i)}$$

为了计算注意力权重，咱们需要一个打分函数，这个函数的作用是衡量两个元素之间的相似度。常用的打分函数是点积，即两个向量对应元素相乘然后求和。

$$e_{ij} = x_i^T x_j$$

In [2]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image015.png)

这里我们将输入元素${x^{(2)}}$叫作query,为什么叫作query，留到下回分解。

我们先来看下中间一步的计算，计算attention score, 即query ${x^{(2)}}$ 与其他输入元素的点积。

点积计算公式如下：
$$e_{ij} = x_i^T x_j$$

也就是两个向量的元素对应相乘然后求和。

In [3]:
query = inputs[1]
atten_scores_2 = torch.empty(inputs.shape[0])
for i in range(inputs.shape[0]):
    atten_scores_2[i] = torch.dot(query, inputs[i])
print(atten_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += element * query[idx]
print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


点积其实是衡量两个向量相似度的方法之一，点积越大，两个向量越相似。这里使用到点积来衡量一个文本序列里，不同token之间的关系，点积越大，关系越密切。
![](./image.png)

下一步就是通过归一化attention score，得到注意力权重attention weight。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image017.png)

归一化公式如下：
$$a_{ij} = \frac{{w_{ij}}}{\sum_{k=1}^{T} w_{ik}}$$

归一化的目的是让注意力权重之和为1，这样就可以保证所有注意力权重加起来等于1。归一化对可解释性和训练稳定性都有好处。

In [5]:
atten_weights_2_tmp = atten_scores_2 / atten_scores_2.sum()
print("Attention weights:",atten_weights_2_tmp)
print("Sum:",atten_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


在实际工程中，使用softmax函数来做归一化更为常见，这种方法对于极大或者极小的值更为鲁棒，同时还能提供更有利的梯度特性。softmax函数公式如下：

$$a_{ij} = \frac{{e^{w_{ij}}}}{\sum_{k=1}^{T} e^{w_{ik}}}$$

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
atten_weights_2_naive = softmax_naive(atten_scores_2)
print("Attention weights:",atten_weights_2_naive)
print("Sum:",atten_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


softmax方法可以确保attention weight总是正的，而除以sum的方法无法保证这一点。这点可以让attention weight可以看作概率分布或者相对重要性。

softmax_naive方法的缺点是，对于极大或者极小的值，计算结果不稳定，容易出现数值溢出。因此建议使用PyTorch自带的softmax函数。

In [7]:
atten_weights_2 = torch.softmax(atten_scores_2, dim=0)
print("Attention weights:",atten_weights_2)
print("Sum:",atten_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image019.png)

最后一步就是计算上下文向量context vector。

计算公式如下：

$$z^{(2)} = \sum_{i=1}^{T} a_{2i} x^{(i)}$$

每一个输入向量乘以对应的attention weight，然后求和，得到上下文向量。

In [8]:
query = inputs[1]
context_vector_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vector_2 += atten_weights_2[i] * x_i
print("Context vector:",context_vector_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])


#### 3.2 Compuing attention weights for all input tokens

为了计算所有输入元素的上下文向量，咱们需要对每个输入元素重复上述步骤。

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-11.png)

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-12.png)

我们重复上述步骤，得到所有输入元素的上下文向量。


In [9]:
atten_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        atten_scores[i, j] = torch.dot(x_i, x_j)
print(atten_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [14]:
# 上面是通过for循环来进行计算的，性能其实很低下，我们可以使用矩阵乘法来优化
# inputs是(6, 3)的矩阵，inputs.T是(3, 6)的矩阵，所以atten_scores是(6, 6)的矩阵
atten_scores = inputs @ inputs.T
print(atten_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [15]:
#接下来咱们把atten_scores归一化，获得attention weight
#dim=-1表示在最后一个维度上进行归一化（这里我就写一个了“#”，cursor就知道我要说明dim的含义）
atten_weights = torch.softmax(atten_scores, dim=-1)
print(atten_weights)


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [17]:
# 验证下每一行和是否为1
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print(row_2_sum)
print("All row sums:", atten_weights.sum(dim=-1))

1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [18]:
# 最后一步，通过矩阵乘法计算context vector
# atten_weights是(6, 6)的矩阵，inputs是(6, 3)的矩阵，所以all_context_vectors是(6, 3)的矩阵
all_context_vectors = atten_weights @ inputs
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [20]:
# 看看上节课计算的context vector
print("Previous 2nd context vector:",context_vector_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


接下来咱们开始增加可训练权重，看看如何实现一个完整的自注意力机制。

#### 4.Implementing self-attention with trainable weights

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-13.png)

接下来真正开启自注意力机制的实现，这也是GPT-2和GPT-3的核心，也是其他类似大模型的核心机制，通过上面的学习，可以知道自注意力机制也叫做缩放点积注意力机制(scaled dot-product attention)。

带可训练权重的自注意力机制和之前相比，最重要的区别是权重矩阵在训练过程中是可以变化的。 这些权重矩阵之所以重要是因为他们可以产生“好的”的context vectors。

接下来咱们通过两个小节来介绍，第一个小节和之前一样，一步步的来把自注意力机制的各个部分实现出来，第二小节会把把这些代码封装成一个python类，以便可以用于大模型的框架中。

#### 4.1一步步计算注意力权重

这里首先思考下什么是可训练权重？
我们先介绍三个可训练权重矩阵，分别是query矩阵($W_q$)，key矩阵($W_k$)和value矩阵($W_v$)。这三个矩阵被用来将嵌入式输入tokensz转为query,key和value向量。如下图所示：
![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-14.png)

类似上节课所讲的，我们将$x^{(2)}$作为query input,query vector为$q^{(2)}$，计算公式如下：
$$q^{(2)} = x^{(2)} W_q$$

同理，key vector和value vector的计算公式如下：
$$k^{(2)} = x^{(2)} W_k$$
$$v^{(2)} = x^{(2)} W_v$$

就像上节课所讲，先计算$x^{(2)}$对应的context vector,然后再计算所有输入元素的context vector。

In [22]:
# 先定义一些变量
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

这里，输出变量的维度d_out和输入变量的维度d_in可以不同，也可以相同。GPT的输入是token embedding，维度是d_in，输出的context vector维度也是d_in。

In [23]:
# 初始化query,key和value权重矩阵
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out),requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out),requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out),requires_grad=False)

torch.nn.Parameter是PyTorch中的一个类，用于创建可训练的参数。torch.nn.Parameter是torch.Tensor的子类，它与torch.Tensor的区别在于，torch.nn.Parameter的默认requires_grad属性为True，而torch.Tensor的默认requires_grad属性为False。这意味着，当使用torch.nn.Parameter创建一个张量时，PyTorch会自动将其标记为需要梯度，以便在反向传播过程中进行优化。
 
这里设置requires_grad=False，表示这些权重矩阵在训练过程中不会被更新。是为了减小计算量，但在模型训练中，我们需要设置requires_grad=True，以便可以更新这些权重矩阵。

In [25]:
#计算query,key和value向量
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print("Query:",query_2)
print("Key:",key_2)
print("Value:",value_2)

Query: tensor([0.4306, 1.4551])
Key: tensor([0.4433, 1.1419])
Value: tensor([0.3951, 1.0037])


这里再说明下权重参数和上节课讲的attention weight的区别，权重参数是可训练的，而attention weight是可训练权重经过softmax归一化后的结果,attention weight决定了每个输入元素对当前元素的贡献度。

In [26]:
#计算所有输入元素对应的keys和values
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:",keys.shape)
print("values.shape:",values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


可以看到我们成功的把输入的embedding从3维度转换到了2维度。

下一步就是计算attention score，即query和key的点积。如下图所示：

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-15.png)

和简化版自注意力机制不同一点是，attention score不是通过输入元素间的点积获得的，而是通过query和key的点积获得的。


首先先计算attention score $w_{22}$

In [27]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [29]:
#计算所有的attention_scores通过矩阵相乘
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


可以看到，attn_score_2第二个元素和attn_score_22一样的，都是1.8524

下一步就是从attention scores获得attention weights,上节课咱们直接使用pytorch提供的softmax方法来将attention scores转换为attention weights，但这里我们新增了一个环节，就是在使用softmax方法之前，先对attention scores进行缩放，也就是attention scores除以输入向量维度的平方根。

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-16.png)


In [30]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5,dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSzJosiqrYDtp_Tv6cQWh8hiqORrOZcjTF-jbKefyLUZlWCRadPnxgNQOdu_nfE9Fzql30&usqp=CAU)

为什么这么做呢？大家可以看下softmax函数的图形，发现x自变量的值越大，softmax的值变化越小，也就是梯度越小，这时候就会带来梯度消失的问题，导致无法训练成功后。

当嵌入维度（embedding dimension）很大时（例如GPT等大型语言模型通常使用的维度大于1000），点积（dot products）可能会变得非常大。这会导致在反向传播过程中，经过softmax函数处理后的梯度变得非常小。

为了解决这个问题，自注意力机制中的点积被缩放，通常是除以嵌入维度的平方根。这样做可以防止点积过大，从而避免softmax函数输出的梯度过小。

由于这种缩放操作，自注意力机制也被称为缩放点积注意力（scaled-dot product attention）。这种缩放有助于保持梯度在一个合理的范围内，从而有助于模型更有效地学习。

通过这种缩放，可以改善训练性能，使得模型在训练过程中能够更稳定地收敛，并且可以更快地学习到数据中的复杂模式。

至于为什么选用嵌入维度的平方根呢？这个缩放因子是基于实验和经验得出的。在实践中，它被证明能够提高模型的训练效率和性能。从信息论的角度来看，缩放点积可以帮助保持信息的方差，从而有助于模型学习到更丰富的表示。理论上，可以使用其他缩放因子，例如嵌入维度本身或其他常数。但是，嵌入维度的平方根被广泛认为是一个好的选择，因为它在实践中表现良好。

这也就是为什么大模型很多时候靠实践的，是不是有点像中医的理论？

![](https://drek4537l1klr.cloudfront.net/raschka/Figures/3-17.png)

下一步就是计算context vectors

和上节课类似，context vectors是attention weights作为权重，乘以values vector,而不是输入向量了。

注意力权重决定了每个值向量在计算上下文向量时的相对重要性。这些权重是通过模型学习得到的，它们反映了模型对序列中不同部分的关注程度。


In [32]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])
